In [34]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [2]:
%pwd

'd:\\srinu_babu_folder\\Deep Learning\\Projects\\CNN Project structure\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'd:\\srinu_babu_folder\\Deep Learning\\Projects\\CNN Project structure'

In [8]:
from ensure import ensure_annotations
from pathlib import Path
from box import ConfigBox
@ensure_annotations
def read_yaml(path_to_yaml:Path):
    
    with open(path_to_yaml) as yaml_file:
        content = yaml.safe_load(yaml_file)
        return ConfigBox(content)

In [9]:
from CNNclassifier.constants import *
from CNNclassifier.utils.utils import read_yaml, create_directory


In [35]:

class ConfigurationManager:
    def __init__(self,
                config_filepath = CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directory([self.config.artifacts_root])
    
    def  get_data_ingestion_config(self):
        config =self.config.data_ingestion
        create_directory([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir = config.unzip_dir

        )
        return data_ingestion_config

In [43]:
import urllib.request as request
from zipfile import ZipFile
from tqdm import tqdm
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config 
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            
            filename, header = request.urlretrieve(
                                    url = self.config.source_URL,
                                    filename = self.config.local_data_file
                                )
        
        

    def get_updated_list_of_files(self,list_of_files):
        return [f for f in list_of_files if f.endswith(".jpg") or f.endswith(".jpeg")]
        

    def preprocessor(self,zf,file,working_dir):
        target_filepath = os.path.join(working_dir,file)
        if not os.path.exists(target_filepath):
            zf.extract(file, working_dir)

    def unzip_and_clean(self):
        with ZipFile(file = self.config.local_data_file,mode="r") as zf:
            list_of_files = zf.namelist()
            updated_list_of_files = self.get_updated_list_of_files(list_of_files)
            for f in tqdm(updated_list_of_files):
                self.preprocessor(zf, f,self.config.unzip_dir)

In [44]:
config = ConfigurationManager()
data_ingestion_config = config.get_data_ingestion_config()

data_ingestion = DataIngestion(config=data_ingestion_config)

data_ingestion.download_file()

data_ingestion.unzip_and_clean()


[2023-12-08 16:49:15,986 - INFO - utils]: yaml file reade at path: configs\config.yaml
[2023-12-08 16:49:15,991 - INFO - utils]: yaml file reade at path: params.yaml
[2023-12-08 16:49:15,995 - INFO - utils]: create directory at: artifacts
[2023-12-08 16:49:15,997 - INFO - utils]: create directory at: artifacts/data_ingestion


100%|██████████| 25000/25000 [00:50<00:00, 499.49it/s]


In [28]:
%pwd

'd:\\srinu_babu_folder\\Deep Learning\\Projects\\CNN Project structure'